In [1]:
'''
    将lv2数据中选取能够与探空气球放飞时间吻合的时间段
    北京时 = 世界时 + 8
    微波辐射计采用北京时，气球放飞的时间是00时和12时，所以微波选用08时和20时
'''
import pandas as pd 

Level_2_path = r'I:\Data\Personal Data\graduation project\SACOL\microwave\2007\200712'
Target_str = 'lv2'

Output_paths = []
for root,dirs,files in os.walk(Level_2_path):
    for file in files:
        if(Target_str in file):
            file_path = os.path.join(root,file)
            Output_paths.append(file_path)

In [2]:
# 创建一个空的DataFrame表：Output
a = pd.read_csv(Output_paths[0])
Del_columns = a.columns[2:10]
a.drop(Del_columns,axis=1,inplace=True)
a.drop('Record',axis=1,inplace=True)
Mean_Output = a.iloc[0:0,:]

In [3]:
def data_op(filepath):
    # 打开数据文件
    data = pd.read_csv(filepath)
    parameter = 11
    a = data.loc[data['10']==parameter]
    Del_columns = data.columns[2:10]
    a.drop(Del_columns,axis=1,inplace=True)
    a.drop('Record',axis=1,inplace=True)
    a = a.iloc[:,0:48]

    # 搜索标志
    Search_points = ['07:30','19:30']
    Output = a.iloc[0:0,:]
    Mean_Output = Output
    for Search_point in Search_points:
        i = 0
        point = 0
        while i < len(a['Date/Time']):
            if( Search_point in str(a['Date/Time'][i:i+1].values)):
                point = i
                break
            i = i +1
        if(point == 0):
            continue
        while((i-point)<60):
            Output = Output.append(a.iloc[i:i+1,:],ignore_index=True)
            if((i - point) == 30):
                Date = a.iloc[i:i+1,0:1]
            i = i + 1
        
        Output = Output.append(Output.mean(),ignore_index=True)
        Output.loc[60,'Date/Time'] = '20' + str(Date.values)[9:11] + str(Date.values)[3:5] + str(Date.values)[6:8] + '_WT' + str(Date.values)[12:14] + str(Date.values)[15:17]
        # print(Output.loc[60,'Date/Time'])
        Mean_Output = Mean_Output.append(Output.loc[60,],ignore_index=True)
    return(Mean_Output)

In [4]:
for Output_path in Output_paths:
    tt = data_op(Output_path)
    Mean_Output = Mean_Output.append(tt)
filename = Level_2_path + '.csv'
Mean_Output.to_csv(filename,index=0)